# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time
import numpy as np

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [38]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/single_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [39]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, 2:]

In [40]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

In [41]:
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))

#for n in range(0, 13):
    #training_rainfall[:, n] = (training_rainfall[:, n] - np.mean(training_rainfall[:, n])) / np.std(training_rainfall[:, n])
    #training_rainfall[:, n] = (training_rainfall[:, n] - np.min(training_rainfall[:, n])) / (np.max(training_rainfall[:, n]) - np.min(training_rainfall[:, n]))

### 1.1 DEPRECATED

This following section uses two separate datasets. Which is not required.

In [29]:
#training_datafile = "E:/31-12-2020/prepared-data/single_train.npy"
#validation_datafile = "E:/31-12-2020/prepared-data/single_valid.npy"
#
#training_rainfallfile = "E:/31-12-2020/prepared-data/expected_train.npy"
#validation_rainfallfile = "E:/31-12-2020/prepared-data/expected_valid.npy"

In [30]:
#training_videos = np.load(training_datafile)
#validation_videos = np.load(validation_datafile)
#
#training_rainfall = np.load(training_rainfallfile)[:, 2:]
#validation_rainfall = np.load(validation_rainfallfile)[:, 2:]

In [31]:
# We need them in X, Y, COLOURS
#training_videos = np.swapaxes(training_videos, 1, 2)
#training_videos = np.swapaxes(training_videos, 2, 3)
#
#validation_videos = np.swapaxes(validation_videos, 1, 2)
#validation_videos = np.swapaxes(validation_videos, 2, 3)

In [32]:
# Scale the data
#training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
#training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))
#
#validation_videos[:, :, :, 0] = (validation_videos[:, :, :, 0] - np.min(validation_videos[:, :, :, 0])) / (np.max(validation_videos[:, :, :, 0]) - np.min(validation_videos[:, :, :, 0]))
#validation_videos[:, :, :, 1] = (validation_videos[:, :, :, 1] - np.min(validation_videos[:, :, :, 1])) / (np.max(validation_videos[:, :, :, 1]) - np.min(validation_videos[:, :, :, 1]))

## 2. Model Definition

Next, a CNN model architecture is defined.

In [50]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    #model.add(Conv2D(16, (2, 2), input_shape=input_shape, kernel_initializer=initializers.Ones()))
    model.add(Conv2D(16, (2, 2), input_shape=input_shape, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(13, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=1, seed=None)))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [51]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    history = model.fit(
        xdata,
        ydata,
        batch_size=2,
        validation_split=0.2,
        callbacks=[tensorboard],
        epochs=50
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [52]:
# Flexible parameters
learning_rate = 0.0001

In [ ]:
# Run each model multiple times
for i in range(0, 5):
    run_name = "TEST-SL_16_16_16__13__22_t-{}".format(int(time.time())) # "SL_16_16_16__13__22_t-{}".format(int(time.time()))
    tb = TensorBoard(log_dir=".\\logs\\{}".format(run_name))
    model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
    history = train_model(run_name, tb, model, training_videos, training_rainfall)
    print("Final loss: {}".format(history.history["val_loss"][-1]))

Epoch 1/50
110/110 [==============================] - 6s 46ms/step - loss: 7781.9374 - val_loss: 7750.2769
Epoch 2/50
110/110 [==============================] - 2s 22ms/step - loss: 4973.2500 - val_loss: 4918.3154
Epoch 3/50
110/110 [==============================] - 2s 20ms/step - loss: 3593.7988 - val_loss: 3620.4668
Epoch 4/50
110/110 [==============================] - 2s 20ms/step - loss: 2719.9181 - val_loss: 3115.1201
Epoch 5/50
110/110 [==============================] - 2s 19ms/step - loss: 2142.8862 - val_loss: 2864.2007
Epoch 6/50
110/110 [==============================] - 2s 20ms/step - loss: 2293.9282 - val_loss: 2731.3508
Epoch 7/50
110/110 [==============================] - 2s 20ms/step - loss: 2170.9590 - val_loss: 2651.0527
Epoch 8/50
110/110 [==============================] - 2s 20ms/step - loss: 2091.8484 - val_loss: 2619.6599
Epoch 9/50
110/110 [==============================] - 2s 21ms/step - loss: 2161.0407 - val_loss: 2561.0305
Epoch 10/50
110/110 [================

110/110 [==============================] - 2s 20ms/step - loss: 1963.2184 - val_loss: 2224.3252
Epoch 26/50
110/110 [==============================] - 2s 20ms/step - loss: 1844.8477 - val_loss: 2217.6616
Epoch 27/50
110/110 [==============================] - 2s 20ms/step - loss: 1784.4672 - val_loss: 2203.1650
Epoch 28/50
110/110 [==============================] - 2s 20ms/step - loss: 1865.7498 - val_loss: 2276.8708
Epoch 29/50
110/110 [==============================] - 2s 20ms/step - loss: 1874.2506 - val_loss: 2209.4475
Epoch 30/50
110/110 [==============================] - 2s 20ms/step - loss: 1595.1468 - val_loss: 2220.4023
Epoch 31/50
110/110 [==============================] - 2s 20ms/step - loss: 1898.5232 - val_loss: 2192.6465
Epoch 32/50
110/110 [==============================] - 2s 20ms/step - loss: 1631.0497 - val_loss: 2185.3735
Epoch 33/50
110/110 [==============================] - 2s 20ms/step - loss: 2004.1585 - val_loss: 2186.3557
Epoch 34/50
110/110 [===================